In [1]:
!pip install pyDOE

  Preparing metadata (setup.py) ... done
  Created wheel for pyDOE: filename=pyDOE-0.3.8-py3-none-any.whl size=18170 sha256=c3a216a00dffdf50332999721d0d25ac60ba64c75d88003abb810874e72f7554
  Stored in directory: /root/.cache/pip/wheels/84/20/8c/8bd43ba42b0b6d39ace1219d6da1576e0dac81b12265c4762e
Successfully built pyDOE


In [2]:
import torch
import torch.nn as nn
import numpy as np
from scipy.io import loadmat
from tqdm import tqdm
from pyDOE import lhs
import torch.optim as optim

In [ ]:
# !git clone https://github.com/broccubali/NoisyICML.git

Cloning into 'NoisyICML'...
remote: Enumerating objects: 320, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 320 (delta 49), reused 64 (delta 16), pack-reused 202 (from 1)
Receiving objects: 100% (320/320), 88.75 MiB | 32.71 MiB/s, done.
Resolving deltas: 100% (142/142), done.


In [14]:
# again.
import torch.nn.init as init
class PINN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList(
            [
                nn.Linear(input_size if i == 0 else hidden_size, hidden_size)
                if i % 2 == 0
                else nn.Tanh()
                for i in range(20)
            ]
        )
        self.layers.append(nn.Linear(hidden_size, output_size))
        for layer in self.layers:
            if isinstance(layer, nn.Linear):
                init.xavier_normal_(layer.weight)  # Initialize weights
                if layer.bias is not None:  # Initialize biases to zero
                    init.zeros_(layer.bias)

        # Trainable parameter for the wave number squared (k^2)
        self.a1 = nn.Parameter(torch.tensor([np.random.rand(1)], dtype=torch.float32, device="cuda"))
        self.a2 = nn.Parameter(torch.tensor([np.random.rand(1)], dtype=torch.float32, device="cuda"))
        # Optimizer
        self.optimizer = torch.optim.LBFGS(
            self.parameters(),
            lr=1.0,  # Larger learning rate for L-BFGS
            max_iter=5000,  # Maximum number of iterations per optimization step
            history_size=100,  # Size of the history buffer
            line_search_fn="strong_wolfe"  # Line search algorithm
        )
        self.loss = nn.MSELoss()

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def loss_fn(self, train, u):
        u_pred = self.forward(train)
        return self.loss(u_pred, u)

    def residual_loss(self, train):
        # Physics-informed loss based on the Helmholtz equation
        g = train.clone()
        g.requires_grad = True
        u_pred = self.forward(g)

        # Compute the gradients for second derivatives (u_xx and u_tt)
        u_diff = torch.autograd.grad(
            u_pred, g, torch.ones_like(u_pred), create_graph=True, retain_graph=True
        )[0]
        u_x = u_diff[:, 0]
        u_y = u_diff[:, 1]
        u_xx = torch.autograd.grad(
            u_x, g, torch.ones_like(g[:, 0]), create_graph=True
        )[0][:, 0] # <---- Similar modification to select the second derivative with respect to X
        u_yy = torch.autograd.grad(
            u_y, g, torch.ones_like(g[:, 1]), create_graph=True
        )[0][:, 1]
        k2 = (self.a1**2 + self.a2**2) * np.pi**2
        fhat = -k2 * torch.sin(self.a1 * np.pi * g[:, 0]) * torch.sin(self.a2 * np.pi * g[:, 1])
        residual = u_xx + u_yy + ((k2) * u_pred) - fhat # <---- Also fixed k to k2
        return self.loss(residual, torch.zeros_like(residual))

    def total_loss(self, train, utrain, flag):
        alpha_female = 10.0
        data_loss = self.loss_fn(train, utrain)  # Match observed data
        physics_loss = self.residual_loss(train)  # Enforce governing equations
        if flag:
            return data_loss, physics_loss, data_loss + physics_loss
        else:
            return data_loss + physics_loss


    def train_model(self, train, utrain, epochs=100):
        for epoch in tqdm(range(epochs)):
            # Closure function for L-BFGS
            def closure():
                self.optimizer.zero_grad()
                loss = self.total_loss(train, utrain, False)
                loss.backward()
                return loss

            # L-BFGS optimization step
            self.optimizer.step(closure)

            # Logging
            if epoch % 100 == 0:
                a, b, loss = self.total_loss(train, utrain, True)
                print(
                    f"Epoch {epoch}, Loss {loss.item()}, "
                    f"a1 {self.a1.item()}, a2 {self.a2.item()}, "
                    f"data loss {a.item()}, physics loss {b.item()}"
                )

In [4]:
x = np.linspace(-1, 1, 201)
y = np.linspace(-1, 1, 201)
X, Y = np.meshgrid(x, y)

# True wave numbers
a1_true, a2_true = 1.0, 1.0
u_true = np.sin(a1_true * np.pi * X) * np.sin(a2_true * np.pi * Y)  # True solution
u_true.shape

(100, 100)

In [5]:
torch.cuda.empty_cache()

In [6]:
device = "cuda"
X_train = torch.tensor(X.flatten(), dtype=torch.float32).to(device)
Y_train = torch.tensor(Y.flatten(), dtype=torch.float32).to(device)
U_train = torch.tensor(u_true.T.flatten(), dtype=torch.float32).to(device)
train = torch.stack((X_train, Y_train), dim=1).to(device)
train.shape

torch.Size([10000, 2])

In [16]:
idx = np.random.choice(train.shape[0], 10000, replace=False)
train = train[idx, :]
U_train = U_train[idx]

In [17]:
model = PINN(input_size=2, hidden_size=20, output_size=1).to(device)
model.train_model(train, U_train, epochs=20000)

  0%|          | 0/20000 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([10000])) that is different to the input size (torch.Size([10000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  0%|          | 1/20000 [00:02<15:29:31,  2.79s/it]

Epoch 0, Loss 0.2450261414051056, a1 0.22708925604820251, a2 -0.0018968358635902405, data loss 0.24502511322498322, physics loss 1.0324930599381332e-06


  1%|          | 101/20000 [00:36<1:52:31,  2.95it/s]

Epoch 100, Loss 0.2450261414051056, a1 0.22708925604820251, a2 -0.0018968358635902405, data loss 0.24502511322498322, physics loss 1.0324930599381332e-06


  1%|          | 120/20000 [00:43<2:00:18,  2.75it/s]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation
from tqdm import tqdm

def visualize_helmholtz(X, Y, data, path):
    """
    Visualizes the 2D Helmholtz solution as an animation.

    Args:
    X, Y : Meshgrid coordinates for plotting
    data : 3D NumPy array [time, X, Y] representing the solution at different times
    path : Path to save the animation
    """
    fig, ax = plt.subplots()

    # Use contour plot or imshow
    im = ax.imshow(data[0], extent=[X.min(), X.max(), Y.min(), Y.max()],
                   origin='lower', cmap='viridis', animated=True)
    ax.set_xlabel("X-axis")
    ax.set_ylabel("Y-axis")
    ax.set_title("Helmholtz Equation Solution Animation")

    def update(frame):
        im.set_array(data[frame])  # Update solution at new time step
        return [im]

    ani = animation.FuncAnimation(fig, update, frames=len(data), interval=50, blit=False)

    writer = animation.PillowWriter(fps=15, bitrate=1800)
    ani.save(path, writer=writer)
    plt.close(fig)

# Example Usage
# Generate example Helmholtz solution data
x = np.linspace(-1, 1, 100)
y = np.linspace(-1, 1, 100)
X, Y = np.meshgrid(x, y)

# Example 2D Helmholtz wave solution evolving over time
time_steps = 50
a1, a2 = 1, 1
data = np.array([np.sin(a1 * np.pi * X) * np.sin(a2 * np.pi * Y) * np.cos(2 * np.pi * t / time_steps)
                 for t in range(time_steps)])

# Visualize and save
# visualize_helmholtz(X, Y, data, "helmholtz_solution.gif")